<a href="https://colab.research.google.com/github/Timang419/deep-learning-self-study-/blob/main/addition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [3]:
# generate the data
data1=np.random.randint(-10,11,size=10000)
data2=np.random.randint(-10,11,size=10000)
x = torch.tensor(np.column_stack((data1, data2))).float()
y = torch.sum(x, axis=1, keepdim=True)



In [4]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [65]:
y.size()

torch.Size([10000, 1])

In [5]:
# print the data
for i in range(5):
  print(x[i],y[i])

tensor([9., 2.]) tensor([11.])
tensor([ 3., -5.]) tensor([-2.])
tensor([ 9., -2.]) tensor([7.])
tensor([-7.,  4.]) tensor([-3.])
tensor([-6.,  5.]) tensor([-1.])


In [6]:
# split the data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

In [7]:
# create the model
def create_model():

  class add_model(nn.Module):
    def __init__(self):
      super().__init__()

      # input layer
      self.input=nn.Linear(2,16)

      # hidden layer
      self.hidden=nn.Linear(16,16)

      # output layer
      self.output=nn.Linear(16,1)

    # forward pass
    def forward(self,x):
      x=F.relu(self.input(x))
      x=F.relu(self.hidden(x))
      x=self.output(x)

      return x

  ADDclassify=add_model()

  # loss function
  lossfun = nn.MSELoss()

  # optimizer
  optimizer = torch.optim.Adam(ADDclassify.parameters(),lr=.05)

  # model output
  return ADDclassify,lossfun,optimizer

In [11]:
# create function to train the model
def train_model(epoch=10):

  # create the model
  net,lossfun,optimizer=create_model()
  net.to(device)

  x_train_device = x_train.to(device)
  y_train_device = y_train.to(device)
  x_test_device = x_test.to(device)
  y_test_device = y_test.to(device)

  # initialize the loss value
  Losses=[]
  Accuracy=[]

  for i in range(epoch):
    # forward pass and loss

      yHat=net(x_train_device)
      loss=lossfun(yHat,y_train_device.view(-1, 1))

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      Losses.append(loss.item())
      Accuracy.append(100*torch.mean((torch.abs(yHat-y_train_device)<1).float()))

  ### test with unseen data
  testAcc=0
  with torch.no_grad():
    prediction=net(x_test_device)
    test_loss=lossfun(prediction,y_test_device.view(-1, 1))
    testAcc=100*torch.mean((torch.abs(prediction-y_test_device)<1).float())

  return Losses,Accuracy,test_loss,testAcc



In [21]:
# test the model once to make sure it runs
AddingMachine,lossfun,optimizer = create_model()
losses,trainacc,testLoss,testacc = train_model(50)

# print the results
losses,testacc
# losses

([74.0376205444336,
  55.84039306640625,
  35.29218292236328,
  15.946361541748047,
  12.157428741455078,
  13.495797157287598,
  10.69914722442627,
  5.868252754211426,
  4.587627410888672,
  6.082798004150391,
  7.019349575042725,
  5.9434123039245605,
  3.704472064971924,
  1.920547366142273,
  1.7377994060516357,
  2.9443817138671875,
  3.9179770946502686,
  3.38214373588562,
  1.896012783050537,
  0.835727870464325,
  0.7821530103683472,
  1.3067498207092285,
  1.7202537059783936,
  1.663307785987854,
  1.217483639717102,
  0.7216300964355469,
  0.4818691313266754,
  0.5581590533256531,
  0.72727370262146,
  0.7222899198532104,
  0.533482015132904,
  0.37554875016212463,
  0.39435315132141113,
  0.5224133729934692,
  0.5936899781227112,
  0.5167753100395203,
  0.34146004915237427,
  0.20117293298244476,
  0.18610990047454834,
  0.25897443294525146,
  0.29673343896865845,
  0.23866434395313263,
  0.14855796098709106,
  0.1174648255109787,
  0.15264731645584106,
  0.1879549324512481

In [14]:
testacc

tensor(100., device='cuda:0')